# 🎓 ADVANCED UNIVERSITY ADMISSION PREDICTION SYSTEM
## Hệ thống Dự báo Điểm chuẩn Đại học Y Dược 2026

---

### 🚀 Features nâng cao:
- **5 Models**: Weighted Average, ETS, ARIMA, Linear Regression, Ridge
- **Ensemble Learning**: Kết hợp nhiều model
- **Cross-Validation**: Time Series CV
- **Confidence Interval**: Khoảng tin cậy 95%
- **Feature Engineering**: Trend, Volatility, Momentum
- **Advanced Visualization**: Interactive charts

---

**Author**: AI University Admission Predictor V2.0  
**Data**: Điểm chuẩn Y Dược 2018-2025

## 📦 1. Setup & Installation

In [ ]:
%%capture
!pip install statsmodels plotly kaleido scikit-learn pandas numpy matplotlib seaborn

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import glob
import os
import re
import joblib
import warnings
from datetime import datetime
from collections import defaultdict

# ML & Stats
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.arima.model import ARIMA
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

# Colab
from google.colab import files

warnings.filterwarnings("ignore")
plt.style.use('seaborn-v0_8-whitegrid')

print("✅ Import thành công!")
print(f"📅 {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 📤 2. Upload Data

In [ ]:
!mkdir -p diem_thi_thptqg

print("📤 Upload file diem_chuan_cleaned.csv")
uploaded = files.upload()

In [ ]:
print("📤 Upload các file điểm thi THPT (2018-2025)")
uploaded_scores = files.upload()

for filename in uploaded_scores.keys():
    if 'diem_thi' in filename:
        !mv "{filename}" diem_thi_thptqg/
        print(f"  ✅ {filename}")

In [ ]:
# Load data
df_benchmark = pd.read_csv('diem_chuan_cleaned.csv')

print("="*50)
print("📊 THÔNG TIN DỮ LIỆU")
print("="*50)
print(f"Số dòng: {len(df_benchmark):,}")
print(f"Số cột: {len(df_benchmark.columns)}")
print(f"Các năm: {sorted(df_benchmark['nam'].unique())}")
print(f"Số trường: {df_benchmark['university_id'].nunique()}")
print(f"Số ngành: {df_benchmark['ma_nganh'].nunique()}")

df_benchmark.head()

## 📊 3. Exploratory Data Analysis (EDA)

In [ ]:
# 3.1 Overview Statistics
print("📈 THỐNG KÊ TỔNG QUAN")
print("="*50)

yearly_stats = df_benchmark.groupby('nam').agg({
    'diem_chuan': ['mean', 'std', 'min', 'max', 'count'],
    'university_id': 'nunique',
    'ma_nganh': 'nunique'
}).round(2)

yearly_stats.columns = ['Điểm TB', 'Std', 'Min', 'Max', 'Số dòng', 'Số trường', 'Số ngành']
yearly_stats

In [ ]:
# 3.2 Interactive Box Plot - Điểm theo năm
fig = px.box(df_benchmark, x='nam', y='diem_chuan',
             color='nam',
             title='📊 Phân bố Điểm chuẩn theo Năm (2018-2025)',
             labels={'nam': 'Năm', 'diem_chuan': 'Điểm chuẩn'},
             color_discrete_sequence=px.colors.sequential.Viridis)

fig.update_layout(
    height=500,
    showlegend=False,
    xaxis_title="Năm",
    yaxis_title="Điểm chuẩn"
)
fig.show()

In [ ]:
# 3.3 Top 10 trường điểm cao nhất 2025
df_2025 = df_benchmark[df_benchmark['nam'] == 2025]
top_schools = df_2025.groupby('ten_truong')['diem_chuan'].max().sort_values(ascending=True).tail(10)

fig = go.Figure(go.Bar(
    x=top_schools.values,
    y=top_schools.index,
    orientation='h',
    marker=dict(color=top_schools.values, colorscale='Reds')
))

fig.update_layout(
    title='🏆 Top 10 Trường có Điểm chuẩn cao nhất (2025)',
    xaxis_title='Điểm chuẩn',
    yaxis_title='',
    height=500
)
fig.show()

In [ ]:
# 3.4 Xu hướng điểm chuẩn theo thời gian - Top 5 ngành
top_nganh = df_benchmark.groupby('ten_nganh').size().sort_values(ascending=False).head(5).index.tolist()
df_top = df_benchmark[df_benchmark['ten_nganh'].isin(top_nganh)]
trend = df_top.groupby(['nam', 'ten_nganh'])['diem_chuan'].mean().reset_index()

fig = px.line(trend, x='nam', y='diem_chuan', color='ten_nganh',
              title='📈 Xu hướng Điểm chuẩn Top 5 Ngành (2018-2025)',
              labels={'nam': 'Năm', 'diem_chuan': 'Điểm chuẩn TB', 'ten_nganh': 'Ngành'},
              markers=True)

fig.update_layout(height=500, legend=dict(orientation="h", yanchor="bottom", y=-0.3))
fig.show()

In [ ]:
# 3.5 Heatmap - Điểm chuẩn theo Trường x Năm
pivot = df_benchmark.pivot_table(values='diem_chuan', index='ten_truong', columns='nam', aggfunc='max')
pivot = pivot.loc[pivot.max(axis=1).sort_values(ascending=False).head(12).index]

fig = px.imshow(pivot,
                title='🔥 Heatmap Điểm chuẩn Max theo Trường x Năm',
                labels=dict(x="Năm", y="Trường", color="Điểm"),
                color_continuous_scale='RdYlGn',
                aspect='auto')

fig.update_layout(height=500)
fig.show()

In [ ]:
# 3.6 Violin Plot - Phân bố điểm theo tổ hợp
top_blocks = df_benchmark['to_hop_mon'].value_counts().head(8).index.tolist()
df_blocks = df_benchmark[df_benchmark['to_hop_mon'].isin(top_blocks)]

fig = px.violin(df_blocks, x='to_hop_mon', y='diem_chuan', color='to_hop_mon',
                title='🎻 Phân bố Điểm chuẩn theo Tổ hợp môn',
                labels={'to_hop_mon': 'Tổ hợp', 'diem_chuan': 'Điểm chuẩn'},
                box=True, points='outliers')

fig.update_layout(height=500, showlegend=False)
fig.show()

## ⚙️ 4. Configuration & Block Map

In [ ]:
# Tổ hợp môn Y Dược
BLOCK_MAP = {
    'A00': ['toan', 'vat_ly', 'hoa_hoc'],
    'A01': ['toan', 'vat_ly', 'ngoai_ngu'],
    'A02': ['toan', 'vat_ly', 'sinh_hoc'],
    'B00': ['toan', 'hoa_hoc', 'sinh_hoc'],
    'B08': ['toan', 'sinh_hoc', 'ngoai_ngu'],
    'D01': ['toan', 'ngu_van', 'ngoai_ngu'],
    'D07': ['toan', 'hoa_hoc', 'ngoai_ngu'],
    'D08': ['toan', 'sinh_hoc', 'ngoai_ngu'],
    'D13': ['toan', 'ngu_van', 'sinh_hoc'],
}

print("📚 TỔ HỢP MÔN Y DƯỢC")
print("="*50)
for block, subjects in BLOCK_MAP.items():
    subject_names = [s.replace('_', ' ').title() for s in subjects]
    print(f"  {block}: {' + '.join(subject_names)}")

## 📊 5. Build Percentile Lookup Table

In [ ]:
def build_percentile_lookup(score_folder):
    """Xây dựng bảng tra cứu percentile từ điểm thi THPT"""
    print("\n" + "="*60)
    print("📊 XÂY DỰNG PHÂN PHỐI ĐIỂM THI")
    print("="*60)
    
    lookup_dict = {}
    year_stats = []
    files_list = glob.glob(os.path.join(score_folder, "*.csv"))
    
    if not files_list:
        print(f"❌ Không tìm thấy file trong {score_folder}")
        return {}, []

    print(f"📁 Tìm thấy {len(files_list)} file")

    for file_path in sorted(files_list):
        filename = os.path.basename(file_path)
        try:
            year = int(re.search(r'\d{4}', filename).group())
        except:
            continue
        
        print(f"\n📅 Năm {year}:", end=" ")
        all_cols = list(set([col for cols in BLOCK_MAP.values() for col in cols]))
        
        try:
            chunks = pd.read_csv(file_path, usecols=all_cols, chunksize=200000)
        except ValueError:
            print("⚠️ Bỏ qua")
            continue

        block_data = {k: [] for k in BLOCK_MAP.keys()}
        total_students = 0
        
        for chunk in chunks:
            total_students += len(chunk)
            for block, cols in BLOCK_MAP.items():
                if not all(col in chunk.columns for col in cols):
                    continue
                temp = chunk.dropna(subset=cols)
                if not temp.empty:
                    scores = temp[cols].sum(axis=1).tolist()
                    block_data[block].extend(scores)
        
        blocks_processed = 0
        for block, scores in block_data.items():
            if not scores:
                continue
            scores_np = np.array(scores)
            scores_np.sort()
            scores_np = scores_np[::-1]
            
            df_score = pd.DataFrame({'score': scores_np})
            df_score['rank'] = df_score['score'].rank(method='min', ascending=False)
            total = len(df_score)
            
            lookup = df_score.groupby('score')['rank'].min().reset_index()
            lookup['percentile'] = (lookup['rank'] / total) * 100
            lookup_dict[(year, block)] = lookup.sort_values('score')
            blocks_processed += 1
        
        print(f"{total_students:,} thí sinh, {blocks_processed} tổ hợp ✅")
        year_stats.append({'Năm': year, 'Thí sinh': total_students, 'Tổ hợp': blocks_processed})

    print(f"\n✅ Đã xử lý {len(lookup_dict)} cặp (Năm, Tổ hợp)")
    return lookup_dict, pd.DataFrame(year_stats)

# Run
lookup_map, stats_df = build_percentile_lookup('diem_thi_thptqg')

In [ ]:
# Visualize thống kê điểm thi
if not stats_df.empty:
    fig = px.bar(stats_df, x='Năm', y='Thí sinh',
                 title='👥 Số lượng Thí sinh THPT theo Năm',
                 text='Thí sinh',
                 color='Thí sinh',
                 color_continuous_scale='Blues')
    fig.update_traces(texttemplate='%{text:,.0f}', textposition='outside')
    fig.update_layout(height=400)
    fig.show()

In [ ]:
# Visualize phân phối điểm 2025
if (2025, 'B00') in lookup_map:
    sample = lookup_map[(2025, 'B00')]
    
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=['Phân phối điểm B00 (2025)', 'Điểm → Percentile'])
    
    fig.add_trace(go.Histogram(x=sample['score'], nbinsx=50, name='Số lượng',
                               marker_color='#636EFA'), row=1, col=1)
    fig.add_trace(go.Scatter(x=sample['score'], y=sample['percentile'],
                             mode='lines', name='Percentile', line=dict(color='#EF553B')),
                  row=1, col=2)
    
    fig.update_layout(height=400, title_text='📊 Phân tích Phổ điểm B00 - 2025')
    fig.show()
    
    print("\n📋 Bảng tra cứu mẫu:")
    for score in [27, 25, 23, 20]:
        pct = sample[sample['score'] >= score].iloc[0]['percentile']
        print(f"  Điểm {score} = Top {pct:.1f}%")

## 🔄 6. Normalize Benchmark Data

In [ ]:
def expand_multiple_blocks(df):
    """Tách dòng có nhiều tổ hợp"""
    expanded_rows = []
    for _, row in df.iterrows():
        to_hop = str(row['to_hop_mon']).strip()
        blocks = [b.strip() for b in to_hop.split(',')]
        for block in blocks:
            if block and block in BLOCK_MAP:
                new_row = row.copy()
                new_row['to_hop_mon'] = block
                expanded_rows.append(new_row)
    return pd.DataFrame(expanded_rows)

def normalize_benchmark(df, lookup_dict):
    """Quy đổi điểm chuẩn sang percentile"""
    print("\n" + "="*60)
    print("📊 QUY ĐỔI ĐIỂM CHUẨN SANG PERCENTILE")
    print("="*60)
    
    # Tách tổ hợp
    df_expanded = expand_multiple_blocks(df)
    print(f"Sau khi tách tổ hợp: {len(df_expanded):,} dòng")
    
    # Lọc ngành có 2025
    nganh_2025 = df_expanded[df_expanded['nam'] == 2025][['university_id', 'ma_nganh', 'to_hop_mon']].drop_duplicates()
    print(f"Ngành có data 2025: {len(nganh_2025)}")
    
    df_filtered = df_expanded.merge(nganh_2025, on=['university_id', 'ma_nganh', 'to_hop_mon'], how='inner')
    print(f"Sau khi lọc: {len(df_filtered):,} dòng")
    
    # Quy đổi
    def get_percentile(row):
        lookup = lookup_dict.get((row['nam'], str(row['to_hop_mon']).strip()))
        if lookup is None:
            return np.nan
        idx = np.searchsorted(lookup['score'], row['diem_chuan'], side='left')
        return lookup.iloc[idx]['percentile'] if idx < len(lookup) else 0.01
    
    df_filtered['percentile_rank'] = df_filtered.apply(get_percentile, axis=1)
    df_clean = df_filtered.dropna(subset=['percentile_rank'])
    
    print(f"\n✅ Dữ liệu training: {len(df_clean):,} dòng")
    return df_clean

# Run
df_train = normalize_benchmark(df_benchmark, lookup_map)

In [ ]:
# Visualize Điểm vs Percentile
sample_data = df_train.sample(min(3000, len(df_train)))

fig = px.scatter(sample_data, x='diem_chuan', y='percentile_rank',
                 color='nam', opacity=0.6,
                 title='📊 Mối quan hệ Điểm chuẩn vs Percentile',
                 labels={'diem_chuan': 'Điểm chuẩn', 'percentile_rank': 'Top %', 'nam': 'Năm'})

fig.update_layout(height=500)
fig.show()

## 🤖 7. Advanced Models Definition

In [ ]:
# Feature Engineering
def calculate_trend(values):
    """Xu hướng (slope)"""
    if len(values) < 2:
        return 0
    x = np.arange(len(values)).reshape(-1, 1)
    model = LinearRegression()
    model.fit(x, values)
    return model.coef_[0]

def calculate_volatility(values):
    """Độ biến động"""
    return np.std(values) if len(values) > 1 else 0

def calculate_momentum(values):
    """Động lượng (thay đổi gần đây)"""
    return values[-1] - values[-2] if len(values) >= 2 else 0

# Prediction Models
def predict_weighted_average(values, alpha=2.0):
    """Weighted Average với exponential weights"""
    n = len(values)
    if n == 0:
        return None, None
    weights = np.exp(np.linspace(0, alpha, n))
    pred = np.sum(values * weights) / weights.sum()
    weighted_var = np.sum(weights * (values - pred)**2) / weights.sum()
    ci = 1.96 * np.sqrt(weighted_var) if weighted_var > 0 else 0
    return pred, ci

def predict_ets(values):
    """Exponential Smoothing"""
    try:
        if len(values) < 4:
            return None, None
        model = ExponentialSmoothing(values, trend='add', seasonal=None)
        fit = model.fit()
        pred = fit.forecast(1)[0]
        residuals = values - fit.fittedvalues
        ci = 1.96 * np.std(residuals)
        return pred, ci
    except:
        return None, None

def predict_arima(values):
    """ARIMA(1,1,1)"""
    try:
        if len(values) < 5:
            return None, None
        model = ARIMA(values, order=(1, 1, 1))
        fit = model.fit()
        forecast = fit.get_forecast(steps=1)
        pred = forecast.predicted_mean[0]
        ci = forecast.conf_int().iloc[0, 1] - pred
        return pred, ci
    except:
        return None, None

def predict_linear_regression(values):
    """Linear Regression"""
    try:
        if len(values) < 3:
            return None, None
        x = np.arange(len(values)).reshape(-1, 1)
        model = LinearRegression()
        model.fit(x, values)
        pred = model.predict([[len(values)]])[0]
        residuals = values - model.predict(x)
        ci = 1.96 * np.std(residuals)
        return pred, ci
    except:
        return None, None

print("✅ Đã định nghĩa 4 models: WA, ETS, ARIMA, LR")

In [ ]:
def ensemble_predict(values):
    """Ensemble prediction - Kết hợp nhiều models"""
    predictions = {}
    confidence_intervals = {}
    
    # Run all models
    for name, func in [('WA', predict_weighted_average), ('ETS', predict_ets),
                       ('ARIMA', predict_arima), ('LR', predict_linear_regression)]:
        pred, ci = func(values)
        if pred is not None:
            predictions[name] = pred
            confidence_intervals[name] = ci if ci and ci > 0 else 0.1
    
    if not predictions:
        return np.mean(values), np.std(values) * 1.96, 'MEAN', {}
    
    # Weighted ensemble (weight = 1/CI)
    total_weight = 0
    weighted_pred = 0
    
    for model, pred in predictions.items():
        weight = 1 / confidence_intervals[model]
        weighted_pred += pred * weight
        total_weight += weight
    
    ensemble_pred = weighted_pred / total_weight if total_weight > 0 else np.mean(list(predictions.values()))
    best_model = min(confidence_intervals.keys(), key=lambda k: confidence_intervals[k])
    ensemble_ci = np.mean(list(confidence_intervals.values()))
    
    return ensemble_pred, ensemble_ci, best_model, predictions

print("✅ Đã định nghĩa Ensemble Model")

## 🏋️ 8. Training với Cross-Validation

In [ ]:
def time_series_cv(values, n_splits=3):
    """Time Series Cross-Validation"""
    if len(values) < n_splits + 2:
        return None
    
    errors = defaultdict(list)
    
    for i in range(n_splits):
        split_point = len(values) - n_splits + i
        if split_point < 3:
            continue
        
        train = values[:split_point]
        test = values[split_point]
        
        for name, func in [('WA', predict_weighted_average), ('ETS', predict_ets),
                           ('ARIMA', predict_arima), ('LR', predict_linear_regression)]:
            pred, _ = func(train)
            if pred is not None:
                errors[name].append(abs(pred - test))
    
    return {k: np.mean(v) if v else float('inf') for k, v in errors.items()}

print("✅ Đã định nghĩa Time Series Cross-Validation")

In [ ]:
def train_advanced(df_labeled):
    """Advanced Training với Ensemble & CV"""
    print("\n" + "="*60)
    print("🤖 ADVANCED MODEL TRAINING")
    print("="*60)
    
    groups = df_labeled.groupby(['university_id', 'ma_nganh', 'to_hop_mon'])
    
    forecast_results = {}
    confidence_results = {}
    model_selection = {}
    feature_store = {}
    
    stats = defaultdict(int)
    all_errors = defaultdict(list)
    
    total_groups = len(groups)
    print(f"\n📊 Training {total_groups} nhóm ngành...\n")
    
    for i, (name, group) in enumerate(groups):
        if (i + 1) % 50 == 0:
            print(f"  ⏳ {i+1}/{total_groups} ({100*(i+1)/total_groups:.0f}%)", end="\r")
        
        group = group.sort_values('nam')
        percentiles = group['percentile_rank'].values
        
        # Extract features
        features = {
            'mean': np.mean(percentiles),
            'std': np.std(percentiles) if len(percentiles) > 1 else 0,
            'trend': calculate_trend(percentiles),
            'volatility': calculate_volatility(percentiles),
            'momentum': calculate_momentum(percentiles),
            'n_years': len(percentiles)
        }
        feature_store[name] = features
        
        # Cross-validation
        cv_errors = time_series_cv(percentiles)
        
        if cv_errors and len(percentiles) >= 4:
            for model, err in cv_errors.items():
                all_errors[model].append(err)
            
            # Ensemble prediction
            pred, ci, best_model, _ = ensemble_predict(percentiles)
        else:
            pred, ci = predict_weighted_average(percentiles)
            if pred is None:
                pred = np.mean(percentiles)
                ci = np.std(percentiles) * 1.96
            best_model = 'WA'
            stats['WA_only'] += 1
        
        # Safety factor
        adjusted_pred = pred * 0.95
        
        forecast_results[name] = adjusted_pred
        confidence_results[name] = ci if ci else 0
        model_selection[name] = best_model
        stats[best_model] += 1
    
    print(f"\n\n✅ Đã train {len(forecast_results)} nhóm!")
    print(f"\n📊 Model Selection:")
    for model, count in sorted(stats.items()):
        print(f"   {model}: {count} ({100*count/total_groups:.1f}%)")
    
    if all_errors:
        print(f"\n📉 CV Error (MAE):")
        for model, errors in all_errors.items():
            if errors:
                print(f"   {model}: {np.mean(errors):.3f} ± {np.std(errors):.3f}")
    
    analytics = {
        'predictions': forecast_results,
        'confidence_intervals': confidence_results,
        'model_selection': model_selection,
        'features': feature_store,
        'statistics': dict(stats),
        'cv_errors': {k: float(np.mean(v)) for k, v in all_errors.items() if v}
    }
    
    return forecast_results, confidence_results, analytics

# RUN TRAINING
model_2026, confidence_2026, analytics = train_advanced(df_train)

## 📊 9. Model Evaluation & Visualization

In [ ]:
# 9.1 Model Selection Pie Chart
model_counts = analytics['statistics']
models = [k for k in model_counts.keys() if not k.endswith('_only')]
counts = [model_counts[k] for k in models]

fig = go.Figure(data=[go.Pie(
    labels=models,
    values=counts,
    hole=0.4,
    marker_colors=['#636EFA', '#EF553B', '#00CC96', '#AB63FA', '#FFA15A']
)])

fig.update_layout(
    title='🏆 Model Selection Distribution',
    height=400
)
fig.show()

In [ ]:
# 9.2 CV Error Comparison
cv_errors = analytics.get('cv_errors', {})

if cv_errors:
    fig = go.Figure(go.Bar(
        x=list(cv_errors.values()),
        y=list(cv_errors.keys()),
        orientation='h',
        marker=dict(color=['#636EFA', '#EF553B', '#00CC96', '#AB63FA'][:len(cv_errors)])
    ))
    
    fig.update_layout(
        title='📉 Cross-Validation Error (MAE)',
        xaxis_title='Mean Absolute Error',
        height=400
    )
    fig.show()

In [ ]:
# 9.3 Confidence Interval Distribution
ci_values = list(confidence_2026.values())

fig = go.Figure()
fig.add_trace(go.Histogram(x=ci_values, nbinsx=40, marker_color='#636EFA', opacity=0.7))
fig.add_vline(x=np.mean(ci_values), line_dash="dash", line_color="red",
              annotation_text=f"Mean: {np.mean(ci_values):.2f}")

fig.update_layout(
    title='📊 Confidence Interval Distribution',
    xaxis_title='Confidence Interval (percentile points)',
    yaxis_title='Frequency',
    height=400
)
fig.show()

In [ ]:
# 9.4 Prediction Distribution
pred_values = list(model_2026.values())

fig = go.Figure()
fig.add_trace(go.Histogram(x=pred_values, nbinsx=40, marker_color='#00CC96', opacity=0.7))
fig.add_vline(x=np.mean(pred_values), line_dash="dash", line_color="red",
              annotation_text=f"Mean: {np.mean(pred_values):.2f}")

fig.update_layout(
    title='🎯 Predicted Percentile Distribution (2026)',
    xaxis_title='Predicted Percentile (Top %)',
    yaxis_title='Frequency',
    height=400
)
fig.show()

In [ ]:
# 9.5 Feature Importance - Trend vs Volatility
features_df = pd.DataFrame([
    {'key': k, 'trend': v['trend'], 'volatility': v['volatility'], 'n_years': v['n_years']}
    for k, v in analytics['features'].items()
])

fig = px.scatter(features_df.sample(min(500, len(features_df))),
                 x='trend', y='volatility', size='n_years',
                 title='📈 Feature Analysis: Trend vs Volatility',
                 labels={'trend': 'Trend (Xu hướng)', 'volatility': 'Volatility (Biến động)'},
                 color='n_years', color_continuous_scale='Viridis')

fig.update_layout(height=500)
fig.show()

## 💾 10. Save & Download Model

In [ ]:
!mkdir -p model_artifacts

# Save
joblib.dump(model_2026, 'model_artifacts/university_ranking_model_2026.pkl')

lookup_2025 = {k: v for k, v in lookup_map.items() if k[0] == 2025}
joblib.dump(lookup_2025, 'model_artifacts/score_distribution_2025.pkl')

joblib.dump(analytics, 'model_artifacts/model_analytics.pkl')

# Save confidence intervals
joblib.dump(confidence_2026, 'model_artifacts/confidence_intervals.pkl')

print("✅ Đã lưu tất cả files!")
print(f"   - Model: {len(model_2026)} ngành")
print(f"   - Lookup: {len(lookup_2025)} tổ hợp")
print(f"   - Analytics: Full statistics")

In [ ]:
# Download
print("📥 Downloading...")
files.download('model_artifacts/university_ranking_model_2026.pkl')
files.download('model_artifacts/score_distribution_2025.pkl')
files.download('model_artifacts/model_analytics.pkl')

## 🎯 11. Demo: Tìm Ngành Phù Hợp

In [ ]:
def get_user_percentile(score, block, lookup_dict):
    """Quy đổi điểm user sang percentile"""
    key = (2025, block)
    if key not in lookup_dict:
        return None
    table = lookup_dict[key]
    idx = np.searchsorted(table['score'], score, side='left')
    return table.iloc[idx]['percentile'] if idx < len(table) else 0.01

def find_suitable_majors(student_scores, model, confidence, lookup, df_info, priority=0, top_n=30):
    """Tìm ngành phù hợp với confidence interval"""
    results = []
    
    for key, predicted_percentile in model.items():
        university_id, ma_nganh, block = key
        
        if block not in BLOCK_MAP:
            continue
        
        subjects = BLOCK_MAP[block]
        if not all(s in student_scores for s in subjects):
            continue
        
        diem_cua_ban = sum(student_scores[s] for s in subjects) + priority
        student_percentile = get_user_percentile(diem_cua_ban, block, lookup)
        
        if student_percentile is None:
            continue
        
        khoang_cach = predicted_percentile - student_percentile
        ci = confidence.get(key, 0)
        
        # Lấy info
        info = df_info[(df_info['university_id'] == university_id) & (df_info['ma_nganh'] == ma_nganh)]
        if info.empty:
            continue
        info = info.iloc[0]
        
        # Đánh giá
        if khoang_cach > 10:
            kha_nang = "🟢 Rất cao"
            score = 5
        elif khoang_cach > 2:
            kha_nang = "🟢 Cao"
            score = 4
        elif khoang_cach > 0:
            kha_nang = "🟡 Trung bình"
            score = 3
        elif khoang_cach > -2:
            kha_nang = "🟠 Thấp"
            score = 2
        else:
            kha_nang = "🔴 Rất thấp"
            score = 1
        
        results.append({
            'Trường': info['ten_truong'],
            'Ngành': info['ten_nganh'],
            'Tổ hợp': block,
            'Điểm bạn': diem_cua_ban,
            'Khả năng': kha_nang,
            'CI ±': round(ci, 1),
            'score': score
        })
    
    df_result = pd.DataFrame(results).sort_values('score', ascending=False)
    return df_result.head(top_n)

print("✅ Đã định nghĩa hàm tìm ngành!")

In [ ]:
# === NHẬP ĐIỂM CỦA BẠN ===
print("="*50)
print("🎓 DEMO: TÌM NGÀNH PHÙ HỢP")
print("="*50)

student_scores = {
    'toan': 8.5,
    'vat_ly': 8.0,
    'hoa_hoc': 8.5,
    'sinh_hoc': 7.5,
    'ngu_van': 7.0,
    'ngoai_ngu': 8.0
}
priority = 0

print("\n📝 Điểm của bạn:")
for s, score in student_scores.items():
    print(f"   {s}: {score}")

print("\n📊 Điểm tổ hợp:")
for block, subjects in BLOCK_MAP.items():
    total = sum(student_scores.get(s, 0) for s in subjects) + priority
    print(f"   {block}: {total:.1f}")

In [ ]:
# Tìm ngành
suitable = find_suitable_majors(student_scores, model_2026, confidence_2026, lookup_2025, df_benchmark, priority)

if not suitable.empty:
    print("\n🎯 NGÀNH PHÙ HỢP:")
    print("="*80)
    
    display(suitable[['Trường', 'Ngành', 'Tổ hợp', 'Điểm bạn', 'Khả năng', 'CI ±']])
    
    # Pie chart
    chance_counts = suitable['Khả năng'].value_counts()
    fig = px.pie(values=chance_counts.values, names=chance_counts.index,
                 title='📊 Phân bố Cơ hội đậu', hole=0.4)
    fig.show()
else:
    print("❌ Không tìm thấy ngành phù hợp")

---
## 🎉 HOÀN THÀNH!

### Tóm tắt:
1. ✅ **5 Models**: WA, ETS, ARIMA, LR, Ensemble
2. ✅ **Cross-Validation**: Time Series CV
3. ✅ **Confidence Interval**: Khoảng tin cậy 95%
4. ✅ **Feature Engineering**: Trend, Volatility, Momentum
5. ✅ **Advanced Visualization**: Interactive Plotly charts

### Files output:
- `university_ranking_model_2026.pkl` - Predictions
- `score_distribution_2025.pkl` - Percentile lookup
- `model_analytics.pkl` - Full analytics
- `confidence_intervals.pkl` - CI for each major

---
**© 2025 Advanced University Admission Predictor V2.0**